<a href="https://colab.research.google.com/github/victorgelpi/SHAPE20_AdvCS1_Day12/blob/master/Copy_of_3608_HW9_Dynamic_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Instructions
1. Copy this notebook to your drive (using the "Copy to Drive" button above)
1. Complete the "TODO" portions of this notebook
1. Test your code using the provided tests (and your own additional tests if you'd like to of course!)
1. Download the notebook in **.ipynb** (**IMPORTANT: note the change of format compared to previous assignments**) format (File -> Download -> Download **.ipynb**)
1. Submit the downloaded **.ipynb** file under the corresponding coding assignments on gradescope. (**IMPORTANT: We will NOT grade .py scripts.**)



# Amino Acid Sequences Alignment and Longest Common Subsequence


## Introduction:

Histone H1 protein binds to linker DNA between nucleosomes forming the macromolecular structure known as the chromatin fiber. Histones H1 are necessary for the condensation of nucleosome chains into higher-order structured fibers. Acts also as a regulator of individual gene transcription through chromatin remodeling, nucleosome spacing and DNA methylation (By similarity). Here we have the protein sequences of Histone H1.2 for 5 different species: human, chimpanzee, mouse, rat and cow.

In this question, we would like use Dynamic Programming to determine the Longest Common Subsequence (LCS) between these protein sequences.

## References for the provided data:
* human: https://www.uniprot.org/uniprot/P16403
* chimp: https://www.uniprot.org/uniprot/H2QSF7
* rat: https://www.uniprot.org/uniprot/P15864
* mouse: https://www.uniprot.org/uniprot/P15864
* cow: https://www.uniprot.org/uniprot/P02253

In [ ]:
## AA sequence data
human = 'MSETAPAAPA AAPPAEKAPV KKKAAKKAGG TPRKASGPPV SELITKAVAA SKERSGVSLA \
     ALKKALAAAG YDVEKNNSRI KLGLKSLVSK GTLVQTKGTG ASGSFKLNKK AASGEAKPKV \
     KKAGGTKPKK PVGAAKKPKK AAGGATPKKS AKKTPKKAKK PAAATVTKKV AKSPKKAKVA \
     KPKKAAKSAA KAVKPKAAKP KVVKPKKAAP KKK'
chimp = 'MSETAPAAPA AAPPAEKAPV KKKAAKKAGG TPRKASGPPV SELITKAVAA SKERSGVSLA \
     ALKKALAAAG YDVEKNNSRI KLGLKSLVSK GTLVQTKGTG ASGSFKLNKK AASGEAKPKV \
     KKAGGTKPKK PVGATKKPKK AAGGATPKKS AKKTPKKAKK PAAATVTKKV AKSPKKAKVA \
     KPKKAAKSAA KAVKPKAAKP KVVKPKKAAP KKK'
mouse = 'MSEAAPAAPA AAPPAEKAPA KKKAAKKPAG VRRKASGPPV SELITKAVAA SKERSGVSLA \
     ALKKALAAAG YDVEKNNSRI KLGLKSLVSK GILVQTKGTG ASGSFKLNKK AASGEAKPQA \
     KKAGAAKAKK PAGAAKKPKK ATGAATPKKA AKKTPKKAKK PAAAAVTKKV AKSPKKAKVT \
     KPKKVKSASK AVKPKAAKPK VAKAKKVAAK KK'
rat = 'MSETAPAAPA AAPPAEKAPA KKKAAKKPAG MRRKASGPPV SELITKAVAA SKERSGVSLA \
     ALKKALAAAG YDVEKNNSRI KLGLKSLVSK GILVQTKGTG ASGSFKLNKK AASGEAKPKA \
     KKAAAAKAKK PAGAAKKPKK ATGAATPKKA AKKTPKKAKK PAAAAVTKKV AKSPKKAKVT \
     KAKKVKSASK AVKPKAAKPK VAKAKKVAAK KK'
cow = 'MSETAPAAPA AAPPAEKTPV KKKAAKKPAG ARRKASGPPV SELITKAVAA SKERSGVSLA \
     ALKKALAAAG YDVEKNNSRI KLGLKSLVSK GTLVQTKGTG ASGSFKLNKK AATGEAKPKA \
     KKAGAAKPKK AAGAAKKTKK ATGAATPKKT AKKTPKKAKK PAAAAVTKKV AKSPKKAKAA \
     KPKKAAKSAA KAVKPKAAKP KVAKPKKAAP KKK'

## Brute force approach

Like we mentioned in class, you could compute all the subsequence of both strings and find the longest one they have in common.

In [ ]:
# Brute force approach
from itertools import combinations

# helper function to find all subsequences to a string S
def all_subsequences(S):
    output = list()
    for i in range(1, len(S) + 1):
        for sub in combinations(S, i):
            output.append(''.join(sub))
    return output

# the real brute force method
def LCS_BF(S,T):
    max = 0
    S_sub, T_sub = all_subsequences(S), all_subsequences(T)
    for sub in S_sub:
        if sub in T_sub and max < len(sub): max = len(sub)
    return max

S = 'abcxe'
T = 'abce'
LCS_BF(S, T)

4

## Problems with the brute force method:
As we mentioned in class, this method is very runtime expensive: A string of length n has $2^n-1$ different possible subsequences (since we do not consider the subsequence with length 0). This implies that the time complexity of the brute force approach will be $O(n * 2^n)$. Thus, for small examples as above, it works fine but for our purpose: to compare the AA sequences that's very long, the expensive runtime becomes a problem.


The following code will not run on your computer because it will not only take too much time but also take all your RAM and then crash.

**You do not need to execute the following call to LCS_BF, but if you try it, you will notice that it is way too slow**

In [ ]:
# don't run this
# on my computer, it ran for 1 minute and crashed because of lack of RAM
LCS_BF(human,chimp)

## Naive recursive approach

Let the input sequences be `S[0,..,m-1]` and `T[0,..,n-1]` of lengths m and n respectively. And let `LCS(S[0,..,m-1], T[0,..,n-1])` be the length of longest common subsequence of the two sequences S and T. The recursive formulas we saw in class are:

**Case 1**: If last characters of both sequences match (or `S[m-1] == T[n-1]`) then
`LCS(S[0,..,m-1], T[0,..,n-1]) = 1 + LCS(S[0,..,m-2], T[0,..,n-2])`

**Case 2**: If last characters of both sequences do not match (or `S[m-1] != T[n-1]`) then
`LCS(S[0,..,m-1], T[0,..,n-1]) = MAX(LCS(S[0,..,m-2], T[0,..,n-1]), LCS(S[0,..,m-1], T[0,..,n-2]))`


In [ ]:
# Naive Recursive approach

def LCS_NR(S, T, i, j):
    # termination condition
    # when we've run through the string
    if i == 0 or j == 0:
        return 0
    # Case 1
    # If last characters of both sequences match (or S[i-1] == T[j-1])
    # then LCS(S,T,i,j) is just 1 + LCS(S,T,i-1,j-1)
    elif S[i-1] == T[j-1]:
        return 1 + LCS_NR(S, T, i-1, j-1)
    # Case 2
    # S[i-1] != T[j-1]""
    # LCS(S,T,i,j) = max(LCS(S, T, i, j-1), LCS(S, T, i-1, j))
    else:
        return max(LCS_NR(S, T, i, j-1), LCS_NR(S, T, i-1, j))

S = 'abcxe'
T = 'abce'
n = len(S)
m = len(T)
LCS_NR(S, T, n, m)

4

## Problems with the naive recursive approach


Again, the following code will not run on your computer because it will not only take too much time but also take all your RAM and then crash.

**You do not need to execute the following call to LCS_NR, but if you try it, you will notice that it is way too slow**

In [ ]:
# don't run this line of code

LCS_NR(human,chimp,len(human),len(chimp))

TODO: In your write up, explain why this recursive approach is very slow.

##Dymanic programming approach


You can test your code on the following small instance.

In [1]:
S = 'abcxe'
T = 'abce'
LCS(S, T)

NameError: name 'LCS' is not defined

## Time Complexity to the DP method
As seen in class, the running time complexity of the dynamic programming algorithm for LCS is $O(mn)$, which is fast enough to handle large instances. In particular, your code should be fast enough to compute the LCS on the provided protein sequences of the five different species, which is not the case of the two other methods provided above.


The results are reported as follows:

In [ ]:
# this line should run instantly
# now you see the power of Dynamic Programming!
print ("The longest common subsequence between the Human's and Chimpanzee's Histone H1.2 protein sequence is of length:", LCS(human, chimp))

In [ ]:
print ("The longest common subsequence between the Human's and Mouse's Histone H1.2 protein sequence is of length:", LCS(human, mouse))
print ("The longest common subsequence between the Human's and Rat's Histone H1.2 protein sequence is of length:", LCS(human, rat))
print ("The longest common subsequence between the Human's and Cow's Histone H1.2 protein sequence is of length:", LCS(human, cow))
print ("The longest common subsequence between the Chimp's and Mouse's Histone H1.2 protein sequence is of length:", LCS(chimp, mouse))
print ("The longest common subsequence between the Chimp's and Rat's Histone H1.2 protein sequence is of length:", LCS(chimp, rat))
print ("The longest common subsequence between the Chimp's and Cow's Histone H1.2 protein sequence is of length:", LCS(chimp, cow))
print ("The longest common subsequence between the Mouse's and Rat's Histone H1.2 protein sequence is of length:", LCS(mouse, rat))
print ("The longest common subsequence between the Mouse's and Cow's Histone H1.2 protein sequence is of length:", LCS(mouse, cow))
print ("The longest common subsequence between the Rat's and Cow's Histone H1.2 protein sequence is of length:", LCS(rat, cow))

TODO: report your results for the above 10 tests in your write up

The results from runnning LCS between the different pairs of protein sequences should indicate that (human, chimp) and (mouse, rat) are the two pairs of animals that have the most similar Histone H1.2 protein sequence. This makes sense since these are pairs of closely related species!